In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import locale
locale.setlocale(locale.LC_ALL, '')
from string import ascii_lowercase
import datetime as dt
from sklearn import linear_model

Hier werden die Covid_Fälle amerikanischer Counties vorbereitet. Jedes County bekommt ein eigenes DF und die DFs werden in einem Dictionary gespeichert. Erstellung der Liste avgwachstum, in der die durchschnittlichen Wachstumsraten der Covid-Zahlen der Counties gespeichert werden.

In [2]:
counties = pd.read_csv('covid_confirmed_usafacts.csv')

counties = counties.drop_duplicates(subset =["County Name"])
counties = counties.drop(['stateFIPS','countyFIPS'], axis=1)
arraycounties = [counties['County Name'].tolist()] + [counties['State'].tolist()]

dfState = pd.DataFrame()
dflistC = counties['County Name'].tolist()
dfState = counties[['County Name','State']]
dfState = dfState.reset_index()
dfState = dfState.drop_duplicates(subset=['County Name'])
dflistC = list(dict.fromkeys(dflistC))
dic = {}

for i in dflistC:
    dic[i] = pd.DataFrame()

k = 0
avgwachstum = []

for name, df in dic.items():
        dic[name] = counties.loc[(counties['County Name'] == arraycounties[0][k]) & (counties['State'] == arraycounties[1][k])]
        dic[name] = dic[name].reset_index()
        dic[name] = dic[name].T
        dic[name] = dic[name].rename(columns={0:'cases'})
        dic[name]['County Name'] = arraycounties[0][k]
        dic[name] = dic[name].iloc[3:]
        dic[name] = dic[name].reset_index()
        dic[name] = dic[name].rename(columns={'index': 'date'})
        dic[name]['date'] = pd.to_datetime(dic[name]['date'], format='%m/%d/%y')
        dic[name] = dic[name].loc[dic[name]['date']< '2020-06-20']
        dic[name]['cases'] =  dic[name]['cases'].astype(int) 
        dic[name]['dailycases'] = dic[name]['cases'].diff()
        dic[name]['dailycases'][dic[name]['dailycases'] < 0] = 0
        dic[name]['Rate'] = dic[name]['cases'].pct_change()
        dic[name]['Rate'] = dic[name]['Rate']*100
        dic[name]['Rate'] = dic[name]['Rate'].replace([np.inf, -np.inf], 0)
        dic[name] = dic[name].fillna(0)
        avgwachstum.append(dic[name]['Rate'].sum()/len(dic[name].index))
        k = k + 1

Vorbereitung der Daten zu den Landflächen und den Einwohnerzahlen amerikanischer Counties.

In [3]:
areacounty = pd.read_excel('LND01.xls')
areacounty = areacounty[['Areaname','LND010190D']]
areacounty['Areaname'] = areacounty['Areaname'].str[:-4]
areacounty['Areaname'] = areacounty['Areaname'] + ' County'
areacounty = areacounty.drop_duplicates(subset =["Areaname"])
areacounty = areacounty.reset_index()

pop = pd.read_csv('covid_county_population_usafacts.csv')

Vorbereitung DF "DFCounty", dort laufen die preprocessed Daten zusammen.

In [4]:
t = {'County':dflistC,'AGR':avgwachstum}
DFCounty = pd.DataFrame(t)
DFCounty['State'] = dfState['State']
DFCounty['Area'] = 0.0

Berechnung der County-Bevölkerungsdichte, da diese nicht abgreifbar war.

In [5]:
#Jedem County wird die passende Fläche zugeordnet
for i in range(len(areacounty)):
    for j in range(len(dflistC)):
        if dflistC[j] == areacounty['Areaname'][i]:
            DFCounty['Area'][j] = areacounty['LND010190D'][i]

DFCounty['Area'] = DFCounty['Area'] * 2.59  #Von Meilen auf Kilometer
DFCounty['Population'] = 0

#Jedem County wird die passende Bevölkerungszahl zugeordnet
for i in range(len(pop)):
    for j in range(len(dflistC)):
        if dflistC[j] == pop['County Name'][i]:
            DFCounty['Population'][j] = pop['population'][i]

#Aus Fläche und Bevölkerung wird Bevölkerungsdichte berechnet, da es diese werde nicht einfach so zum Abruf gab
DFCounty['Density'] = DFCounty['Population'] / DFCounty['Area'] 
DFCounty=DFCounty.replace([np.inf, -np.inf], np.nan)
DFCounty = DFCounty.fillna(0)
DFCounty = DFCounty[DFCounty['Density'] != 0]
DFCounty = DFCounty[DFCounty['AGR'] != 0]
DFCounty = DFCounty.fillna(0)

Covid-Fälle deutscher Landkreise werden vobereitet. und die Zeile täglicher Wachstum wird hinzugefügt. Gespeichert wird dies in dem DF "kreise_zuwachs".

In [6]:
kreise_zuwachs = pd.read_csv('COVID-19 Cases Germany_1604319250294.csv')
kreise_zuwachs.drop(['relative_case_changes','fb_id','cases_per_population','relative_death_changes','bundesland_ags','kreis_ags','kreis_nuts','kreis_name','deaths_per_population','fb_datetime'], axis=1, inplace=True)
kreise_zuwachs = kreise_zuwachs.fillna(0)
kreise_zuwachs['deaths'] = kreise_zuwachs['deaths'].astype(int)
kreise_zuwachs['publication_datetime'] = kreise_zuwachs['publication_datetime'].str[:11]
kreise_zuwachs = kreise_zuwachs.sort_values(['location_label','publication_datetime'])
kreise_zuwachs['täglicher Wachstum'] = kreise_zuwachs['cases'].pct_change()

Das selbe wie bei den Counties wird hier mit den Landkreisen gemacht, jedoch wird eine weitere Variable, die Tode vorbereitet.

In [7]:
arraykreise = kreise_zuwachs['location_label'].tolist()
arraykreise = list(dict.fromkeys(arraykreise))  #Entfernt alle Dubletten
#arraykreise = ['Mettmann','Main-Taunus-Kreis','Offenbach','Esslingen','Recklinghausen','Altmarkkreis Salzwedel','Prignitz','Ostprignitz-Ruppin','Uckermark','Stendal']
arraykreise = sorted(arraykreise)
dflist = arraykreise
d = {} #dict damit können alle erstelleten DataFrames aufgerufen werden

for j in dflist:
   d[j] = pd.DataFrame()
i = 0

durchschnittlicheswachstum = []
durchschnittlicheswachstumD = []

for name, df in d.items():
        d[name] = kreise_zuwachs[kreise_zuwachs['location_label'] == arraykreise[i]]
        d[name] = d[name].drop(columns=['population','bundesland_name'])
        d[name]['dailycases'] = d[name]['cases_per_100k'].diff()
        d[name]['dailydeaths'] = d[name]['deaths_per_100k'].diff()
        d[name]['publication_datetime'] = pd.to_datetime(d[name]['publication_datetime'])
        d[name] = d[name].sort_values(['publication_datetime'])
        d[name] = d[name].loc[d[name]['publication_datetime']< '2020-06-20']
        d[name]['dailycases'][d[name]['dailycases'] < 0] = 0
        d[name]['Rate'] = d[name]['cases'].pct_change()
        d[name]['Rate'] = d[name]['Rate']*100
        d[name]['Rate'] = d[name]['Rate'].replace([np.inf, -np.inf], 0)
        
        d[name]['RateD'] = d[name]['deaths'].pct_change()
        d[name]['RateD'] = d[name]['RateD']*100
        d[name]['RateD'] = d[name]['RateD'].replace([np.inf, -np.inf], 0)
        d[name] = d[name].fillna(0)
        d[name] = d[name].reset_index()
        
        #10 DFs sind vorhanden nun muss durchschnittliche Wachstumsrate berechnet werden
        durchschnittlicheswachstum.append(d[name]['Rate'].sum()/len(d[name].index))
        durchschnittlicheswachstumD.append(d[name]['RateD'].sum()/len(d[name].index))
        i = i + 1

Weitere Bearbeitung einzelner Landkreise und Counties, wie der HSK oder Albany County. Diese DFs lassen sich über das Dictionary sehr einfach über d[Landkreis] aufrufen.

In [8]:
d['Hochsauerlandkreis'].loc[0,'dailycases'] = d['Hochsauerlandkreis'].loc[0, 'cases_per_100k']
d['Hochsauerlandkreis']['kumcases'] = d['Hochsauerlandkreis']['dailycases'].cumsum()

d['Hochsauerlandkreis']['dailydeaths'] = d['Hochsauerlandkreis']['deaths_per_100k'].diff()
d['Hochsauerlandkreis']['dailydeaths'][d['Hochsauerlandkreis']['dailydeaths'] < 0] = 0
d['Hochsauerlandkreis'].loc[0,'dailydeaths'] = d['Hochsauerlandkreis'].loc[0, 'deaths']
d['Hochsauerlandkreis']['kumdeaths'] = d['Hochsauerlandkreis']['dailydeaths'].cumsum()

In [9]:
d['Essen'].loc[0,'dailycases'] = d['Essen'].loc[0, 'cases_per_100k']
d['Essen']['kumcases'] = d['Essen']['dailycases'].cumsum()

d['Essen']['dailydeaths'] = d['Essen']['deaths_per_100k'].diff()
d['Essen']['dailydeaths'][d['Essen']['dailydeaths'] < 0] = 0
d['Essen'].loc[0,'dailydeaths'] = d['Essen'].loc[0, 'deaths_per_100k']
d['Essen']['kumdeaths'] = d['Essen']['dailydeaths'].cumsum()

In [10]:
d['Hochsauerlandkreis']['Wochentag'] = d['Hochsauerlandkreis']['publication_datetime'].dt.day_name()
dic['Albany County']['Weekday'] = dic['Albany County']['date'].dt.day_name()
d['Essen']['Wochentag'] = d['Essen']['publication_datetime'].dt.day_name()

Vorbereitung der Daten zu den Bevölkerungsdichten deutscher Landkreise.

In [11]:
dichte = pd.read_excel('A_BevDichte_Kreise.xlsx')
dichte = dichte.drop(columns = ['Kennziffer','Thünen-Typ'])
dichte = dichte.sort_values(by=['Name'])
dichte = dichte.reset_index()
dichte = dichte.drop(columns = 'index')

#Wird genutzt um die Stadt von dem Landkreis zu trennen
name = ' Stadt'

for j in range(len(dichte)):
    if j == 401:
        break
    else:
        if dichte['Name'][j] == dichte['Name'][j+1]:
            if dichte['Bevölkerungsdichte 2014'][j] > dichte['Bevölkerungsdichte 2014'][j+1]:
                dichte['Name'][j+1] = dichte['Name'][j+1] + name
            else:
                dichte['Name'][j] = dichte['Name'][j] + name

dichteU = pd.DataFrame(columns= ['Landkreis/Kreis','Bev.D.Ew./km²'])
dichteU['Landkreis/Kreis'] = arraykreise
dichteU = dichteU.fillna(0)

for i in range(len(dichte)):
    for j in range(len(arraykreise)):
        if arraykreise[j] == dichte['Name'][i]:
            dichteU['Bev.D.Ew./km²'][j] = dichte['Bevölkerungsdichte 2014'][i]

Bevölkerungsdichte wird den durchschnittlichen Wachstumsraten der Coronainfektionen von den Landkreisen zugeordnet.

In [12]:
t = {'Landkreise/Kreis':arraykreise,'AGR':durchschnittlicheswachstum}
compLK = pd.DataFrame(t) #compLK = Vergleich der Landkreise von Wachstum und Bevölkerungsdichte
compLK['AGR_D'] = durchschnittlicheswachstumD
compLK['Bev.D.Ew./km²'] = dichteU['Bev.D.Ew./km²'].astype(float)
compLK = compLK.fillna(0)
compLK = compLK.loc[compLK['Bev.D.Ew./km²'] != 0.0]
compLK = compLK.loc[compLK['AGR'] != 0.0]
compLK = compLK.loc[compLK['AGR_D'] != 0.0]
#Sonst geht Clustering nicht
compLKfnl = compLK.drop(columns =['Landkreise/Kreis'])
compLKfnl = compLKfnl.rename(columns={'Bev.D.Ew./km²':'Dichte'})
compLKfnl = compLKfnl.fillna(0)

Vorbereitung der töglichen Covid-Zahlen für die 16 deutschen Bundesländer in die DF "bundzuwachsraw".

In [13]:
bundzuwachsraw = pd.read_csv('table-1.csv')
bundzuwachsraw.set_index('Datum')
bundzuwachsraw = bundzuwachsraw.drop(['KW','Differenz zum Vortag','Bem.','Datum'],1) 
bundzuwachsraw = bundzuwachsraw.iloc[:-1]
bundzuwachsraw = bundzuwachsraw.fillna(0)
bundzuwachsraw = bundzuwachsraw.replace('—',0)

for column in bundzuwachsraw:
    bundzuwachsraw[column] = bundzuwachsraw[column].astype(str)
    bundzuwachsraw[column] = bundzuwachsraw[column].str.replace('.','')
    for c in ascii_lowercase:
        bundzuwachsraw[column] = bundzuwachsraw[column].str.replace(c,'')
    bundzuwachsraw[column] = bundzuwachsraw[column].str.replace(')','')
    bundzuwachsraw[column] = bundzuwachsraw[column].str.replace('(','')
    bundzuwachsraw[column] = bundzuwachsraw[column].astype(int)    

bundzuwachs = bundzuwachsraw.diff()
bundzuwachs.iloc[0] = bundzuwachsraw.iloc[0]
bundzuwachsraw['Date'] = pd.date_range(start='24/2/2020', periods=len(bundzuwachsraw), freq='D')        
bundzuwachs['Date'] = pd.date_range(start='24/2/2020', periods=len(bundzuwachsraw), freq='D')
bundzuwachsraw = bundzuwachsraw.set_index('Date')
bundzuwachs = bundzuwachs.set_index('Date')

Erstellung der DF "bundchange", in der die tägliche Veränderung festgehalten wird.

In [14]:
bundchange = bundzuwachsraw.pct_change()
bundchange = bundchange*100
bundchange = bundchange.fillna(0)
bundchange = bundchange.replace([np.inf, -np.inf], 0) 

Aufsplittung der Daten zu den Bundesländern, um mit ihnen Prognosen zu erstellen.

In [15]:
bundzuwachsraw_1 = bundzuwachsraw.iloc[:118] # Februar Tage + März + April + Mai + Juni bis zum 19
bundzuwachsraw_2 = bundzuwachsraw.iloc[118:]
bundzuwachs_1 = bundzuwachs.iloc[:118]
bundzuwachs_2 = bundzuwachs.iloc[118:]
bundchange_1 = bundchange.iloc[:118]
bundchange_2 = bundchange.iloc[118:]

In [16]:
bundzuwachsraw_1['date'] = bundzuwachsraw_1.index
bundzuwachsraw_1['datef'] = pd.to_datetime(bundzuwachsraw_1['date'])
bundzuwachsraw_1['datef']=bundzuwachsraw_1['datef'].map(dt.datetime.toordinal)
bundzuwachsraw_2['date'] = bundzuwachsraw_2.index
bundzuwachsraw_2['datef'] = pd.to_datetime(bundzuwachsraw_2['date'])
bundzuwachsraw_2['datef']=bundzuwachsraw_2['datef'].map(dt.datetime.toordinal)

Prognoseerstellung für die Bundesländer, mit den zuvor vorbereiteten Daten. In "durchwachstumfnl" sind nun die Prognostizierten Werte enthalten.

In [17]:
bundzuwachspred = pd.DataFrame(bundzuwachsraw_2['date'])
column = 'BY'

for column in bundzuwachsraw_1:
    y = bundzuwachsraw_1[column].to_numpy()
    x = bundzuwachsraw_1['datef'].to_numpy()
    xpred = bundzuwachsraw_2['datef'].to_numpy()
    ypred = bundzuwachsraw_2[column].to_numpy()
    x = x.reshape(-1, 1)
    y = y.reshape(-1, 1)
    xpred = xpred.reshape(-1, 1)
    regr = linear_model.LinearRegression()
    regr.fit(x, y)
    predic = regr.predict(xpred)
    bundzuwachspred[column] = predic
bundzuwachspred = bundzuwachspred.drop(columns = ['date','datef','Gesamt'])
bundchangepred = bundzuwachspred.pct_change()
bundchangepred = bundchangepred*100
bundchangepred = bundchangepred.fillna(0)
bundchangepred = bundchangepred.replace([np.inf, -np.inf], 0)
bundchangepred = bundchangepred.drop(bundchangepred.index[0])

durchschnittlichwachstum = bundchangepred.sum()/len(bundchange_2.index)
durchwachstumfnl = pd.DataFrame({'Bundesland':durchschnittlichwachstum.index,'AGR':durchschnittlichwachstum.values})
liste = ['Baden-Württemberg','Bayern','Berlin','Brandenburg','Bremen','Hamburg','Hessen','Mecklenburg-Vorpommern','Niedersachsen','Nordrhein-Westfalen','Rheinland-Pfalz','Saarland','Sachsen','Sachsen-Anhalt','Schleswig-Holstein','Thüringen']

for i, row in durchwachstumfnl.iterrows():
        durchwachstumfnl.at[i,'Bundesland'] = liste[i]